In [ ]:
! pip install kaggle

In [ ]:
# Create .kaggle directory and copy the kaggle.json file
! mkdir %USERPROFILE%\.kaggle 2>nul
! copy /Y kaggle.json %USERPROFILE%\.kaggle\

In [ ]:
! kaggle datasets download -d kazanova/sentiment140

In [ ]:
from zipfile import ZipFile
dataset= 'sentiment140.zip'

with ZipFile (dataset, 'r')as zip:
    zip.extractall()
    print("done! the dataset  is extracted")

In [ ]:
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
import nltk
nltk.download('stopwords')

In [ ]:
print(stopwords.words('english'))

In [ ]:
twitter_data = pd.read_csv('twitter_data.csv', encoding= 'ISO-8859-1')

In [ ]:
column_names = ['target', 'id', 'date', 'flag', 'user', 'text']
twitter_data = pd.read_csv('twitter_data.csv', names=column_names, encoding= 'ISO-8859-1')

In [ ]:
twitter_data.shape

In [ ]:
# checking the missing values

twitter_data.isnull().sum()

In [ ]:
# checking the distribution of target column
twitter_data['target'].value_counts()

In [ ]:
twitter_data.replace({'target':{4:1}}, inplace=True)
twitter_data['target'].value_counts()

##  STEMMING

In [ ]:
port_stem = PorterStemmer()


In [ ]:
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]', ' ', content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)

    
    return stemmed_content

In [ ]:
twitter_data['stemmed_content'] = twitter_data['text'].apply(stemming)

In [ ]:
twitter_data['target']

In [ ]:
# seperating data and label
X = twitter_data['stemmed_content'].values
Y = twitter_data['target'].values

### Splitting the data into training data and testing data


In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=2)

In [ ]:
# converting the textual data to feature vectors
vectorizer = TfidfVectorizer()
X_train= vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)
 

### Training

In [ ]:
model= LogisticRegression(max_iter=1000)

In [ ]:
model.fit(X_train, Y_train)

### Evaluation
#### Accuracy Score

In [ ]:
# Accuracy Score on training data

X_train_prediction= model.predict(X_train)
training_data_accuracy = accuracy_score(Y_train, X_train_prediction)

In [ ]:
print('Accuracy score of training data : ', training_data_accuracy)

In [ ]:
# Accuracy Score on test data

X_test_prediction= model.predict(X_test)    
test_data_accuracy = accuracy_score(Y_test, X_test_prediction)

In [ ]:
print('Accuracy score of testing data : ', test_data_accuracy)

In [ ]:
# Save model

import pickle
filename = 'trained_model.sav'
pickle.dump(model, open(filename, 'wb'))

In [ ]:
# Load the saved model

loaded_model= pickle.load(open('trained_model.sav', 'rb'))

In [ ]:
X_new = X_test[3]
print(Y_test[3])

prediction = model.predict(X_new)
print(prediction)

if prediction[0] == 0:
    print('Negative Tweet')

else:
    print('Positive Tweeet')